In [1]:
!pip3 install wandb

     |████████████████████████████████| 2.0MB 4.0MB/s 
     |████████████████████████████████| 102kB 7.8MB/s 
     |████████████████████████████████| 133kB 17.1MB/s 
     |████████████████████████████████| 163kB 18.8MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=a16841cb3b06f0c3bb5b89a90b6a0dae4e7ec4926dda3273e001b7c3c07615c8
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=8557a419ce35c132a0658a3c04ee6e3f52b59a680b22809429b2a515ad2084bf
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Read in the insurance dataset
insurace = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
# Create a coloum transformer
ct = make_column_transformer(
    (MinMaxScaler(), ['age','bmi','children']), # turn all values in this coloum between 0 and 1
    (OneHotEncoder(handle_unknown='ignore'), ['sex','smoker','region'])
)

# Create X and y
X = insurace.drop('charges',axis=1)
y = insurace['charges']

# Split the data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

# Fit the coloum transformer to training data
ct.fit(X_train)

# Transform training and data with (Min max scaler) and One hot encoder
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [3]:
import wandb
from wandb.keras import WandbCallback
wandb.init(config={"hyper": "parameter"},name='750-20000-Final-2')#,project='test', entity='ranuga-d') # Sigmoid linear softmax Swish ReLU Tanh
model = tf.keras.Sequential([
  tf.keras.layers.Dense(1),
  tf.keras.layers.Dense(4000,activation='swish'),
  tf.keras.layers.Dense(4000,activation='swish'),
  tf.keras.layers.Dense(4000,activation='swish'),
  tf.keras.layers.Dense(4000,activation='swish'),
  tf.keras.layers.Dense(4000,activation='swish'),
  tf.keras.layers.Dense(1),
])
model.compile(loss=tf.keras.losses.mae,metrics=['mae'],optimizer=tf.keras.optimizers.Adam())
model.fit(X_train_normal,y_train,validation_data=(ct.transform(X_test),y_test),callbacks=[WandbCallback(log_weights=True),tf.keras.callbacks.EarlyStopping(patience=25),tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae',factor=0.2,patience=2,min_lr=0.001,verbose=2)],epochs=750)
model.save('/content/drive/MyDrive/Colab Notebooks/models/750-big-dog-model-final-v2.h5')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 1/750
32/32 [==============================] - 21s 609ms/step - loss: 11488.8396 - mae: 11488.8396 - val_loss: 8545.1279 - val_mae: 8545.1279
Epoch 2/750
32/32 [==============================] - 27s 837ms/step - loss: 8202.5498 - mae: 8202.5498 - val_loss: 8440.7168 - val_mae: 8440.7168
Epoch 3/750
32/32 [==============================] - 24s 755ms/step - loss: 8278.0564 - mae: 8278.0564 - val_loss: 8625.5137 - val_mae: 8625.5137
Epoch 4/750
32/32 [==============================] - 29s 896ms/step - loss: 8615.1996 - mae: 8615.1996 - val_loss: 8249.5557 - val_mae: 8249.5557
Epoch 5/750
32/32 [==============================] - 28s 876ms/step - loss: 8735.2320 - mae: 8735.2320 - val_loss: 8490.2656 - val_mae: 8490.2656
Epoch 6/750
32/32 [==============================] - 29s 835ms/step - loss: 8269.3124 - mae: 8269.3124 - val_loss: 7936.0000 - val_mae: 7936.0000
Epoch 7/750
32/32 [==============================] - 30s 934ms/step - loss: 7492.6928 - mae: 7492.6928 - val_loss: 7707.44

KeyboardInterrupt: ignored